This tutorial illustrates a few of the most common utility functions available in the `utils` directory.

# Time Conversions

Time conversion utilities exist between the four modes of GPS Week and time of week in seconds, GPS milliseconds since start of GPS Epoch, Unix milliseconds since start of unix epoch, and Python datetime objects (timezone assumed to be UTC if none provided).

Functionality exists for all 12 combinations between the four time instances, but here we show just one example of looping through each instance time.

In [ ]:
from datetime import datetime, timezone
import gnss_lib_py as glp

Methods to convert `datetime` to the other types of time instances.

In [ ]:
time_now = datetime.now(tz=timezone.utc)
time_now

# convert datetime to GPS week and GPS time of week
gps_week, gps_tow = glp.datetime_to_tow(time_now)
print(f"gps week: {gps_week} gps tow: {gps_tow}")

# convert datetime to GPS milliseconds
gps_millis = glp.datetime_to_gps_millis(time_now)
print(f"GPS milliseconds: {gps_millis}")

# convert datetime to UNIX milliseconds
unix_millis = glp.datetime_to_unix_millis(time_now)
print(f"UNIX milliseconds: {unix_millis}")

Methods to convert GPS week and GPS time of week to the other types of time instances.

In [ ]:
# convert GPS week and GPS time of week to datetime
datetime = glp.tow_to_datetime(gps_week, gps_tow)
print("datetime in UTC: ",datetime.strftime("%d %B, %Y %H:%M:%S"))

# convert GPS week and GPS time to GPS milliseconds
gps_millis = glp.tow_to_gps_millis(gps_week, gps_tow)
print(f"GPS milliseconds: {gps_millis}")

# convert GPS week and GPS time to UNIX milliseconds
unix_millis = glp.tow_to_unix_millis(gps_week, gps_tow)
print(f"UNIX milliseconds: {unix_millis}")

Methods to convert GPS milliseconds to other types of time instances.

In [ ]:
# convert GPS milliseconds to datetime
datetime = glp.gps_millis_to_datetime(gps_millis)
print("datetime in UTC: ",datetime.strftime("%d %B, %Y %H:%M:%S"))

# convert GPS milliseconds to GPS week and GPS time
gps_week, gps_tow = glp.gps_millis_to_tow(gps_millis)
print(f"gps week: {gps_week} gps tow: {gps_tow}")

# convert GPS milliseconds to UNIX milliseconds
unix_millis = glp.gps_to_unix_millis(gps_millis)
print(f"UNIX milliseconds: {unix_millis}")

Methods to convert UNIX milliseconds to other types of time instances.

In [ ]:
# convert UNIX milliseconds to datetime
datetime = glp.unix_millis_to_datetime(unix_millis)
print("datetime in UTC: ",datetime.strftime("%d %B, %Y %H:%M:%S"))

# convert UNIX milliseconds to GPS week and GPS time
gps_week, gps_tow = glp.unix_millis_to_tow(unix_millis)
print(f"gps week: {gps_week} gps tow: {gps_tow}")

# convert GPS milliseconds to UNIX milliseconds
gps_millis = glp.unix_to_gps_millis(unix_millis)
print(f"GPS milliseconds: {gps_millis}")